In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
          CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_manufacturer 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
SELECT 
    a._bic_zc_manftr AS manufacturer_id,  
    a.txtmd AS manufacturer_name
FROM (
    SELECT 
        *, ROW_NUMBER() OVER (PARTITION BY _bic_zc_manftr ORDER BY proc_date DESC) rn 
    FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_zc_manftr
    WHERE end_date = '2400-01-01'
) a 
WHERE rn = 1
""")